In [1]:
%matplotlib widget

import cell_model_base as cs_model
import detect_peaks as dp
import numpy as np
import matplotlib.pyplot as plt
import place_cell_base as pl


In [32]:
mean_video_srate = 100.
total_Time = 500
_lambda = 10

x_coordinates,y_coordinates,speed,timevector = cs_model.generate_randomWalk2(input_srate = mean_video_srate,input_total_Time = total_Time,heading_srate = 10., speed_srate = 5., rho1  = 1,sigma = 0.02,mu_e  = 0.,smooth_coeff = 0.5)
All_arrival_time,I_timestamps = cs_model.generate_arrivals(_lambda,total_Time)

modulated_timestamps = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 20)
# modulated_timestamps1 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)
# modulated_timestamps2 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 10, s = 5)
# modulated_timestamps3 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 90, s = 5)
# modulated_timestamps4 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 90, s = 5)
# modulated_timestamps5 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 10, s = 5)

# modulated_timestamps = np.unique(np.concatenate([modulated_timestamps1,modulated_timestamps2,modulated_timestamps3,modulated_timestamps4,modulated_timestamps5]))
modulated_timestamps = np.random.choice(modulated_timestamps,50,replace=False)

calcium_imag,timevector = cs_model.generate_CalciumSignal(modulated_timestamps,noise_level = 0.01, b = 5.)
calcium_events = modulated_timestamps.shape[0]
I_peaks = dp.detect_peaks(calcium_imag,mpd=0.5*mean_video_srate,mph=1.*np.nanstd(calcium_imag))

plt.figure()
plt.plot(x_coordinates,y_coordinates,'k.')
plt.plot(x_coordinates[I_peaks],y_coordinates[I_peaks],'r.')
plt.show()

# include feature allowing to select multiple positions
# include a feature allowing to draw the 2D enviroment


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.figure()
plt.plot(timevector,calcium_imag,'k')
plt.plot(timevector[I_peaks],calcium_imag[I_peaks],'r.')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:

%matplotlib widget

import cell_model_base as cs_model
import detect_peaks as dp
import numpy as np
import matplotlib.pyplot as plt
import place_cell_base as pl

mean_video_srate = 100.
total_Time = 500
_lambda = 10

x_coordinates,y_coordinates,speed,timevector = cs_model.generate_randomWalk2(input_srate = mean_video_srate,input_total_Time = total_Time,heading_srate = 10., speed_srate = 5., rho1  = 1,sigma = 0.02,mu_e  = 0.,smooth_coeff = 0.5)
All_arrival_time,I_timestamps = cs_model.generate_arrivals(_lambda,total_Time)

modulated_timestamps = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)
# modulated_timestamps1 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)
# modulated_timestamps2 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 10, s = 5)
# modulated_timestamps3 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 90, s = 5)
# modulated_timestamps4 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 90, s = 5)
# modulated_timestamps5 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 10, s = 5)

# modulated_timestamps = np.unique(np.concatenate([modulated_timestamps1,modulated_timestamps2,modulated_timestamps3,modulated_timestamps4,modulated_timestamps5]))
modulated_timestamps = np.random.choice(modulated_timestamps,50,replace=False)

calcium_imag,timevector = cs_model.generate_CalciumSignal(modulated_timestamps,noise_level = 0.01, b = 5.)
calcium_events = modulated_timestamps.shape[0]
I_peaks = dp.detect_peaks(calcium_imag,mpd=0.5*mean_video_srate,mph=1.*np.nanstd(calcium_imag))

plt.figure()
plt.plot(x_coordinates,y_coordinates,'k.')
plt.plot(x_coordinates[I_peaks],y_coordinates[I_peaks],'r.')
plt.show()

place_cell = pl.PlaceCell(RatSession='Sst',day = 1, ch = 1, dataset = 'Exploration',mean_video_srate = 100.,mintimespent =  0.1,minvisits = 1, speed_threshold = 2.5, nbins_cal = 10,
                        num_surrogates = 200,num_cores = 20, nbins_pos_x = 10, nbins_pos_y = 10, placefield_nbins_pos_x = 50, placefield_nbins_pos_y = 50)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
PlaceCellObject.keys()

dict_keys(['signalMap', 'signalMapSmoothed', 'ocuppancyMap', 'x_grid', 'y_grid', 'numb_events', 'events_index', 'mutualInfo_original', 'mutualInfo_zscored', 'mutualInfo_permutation'])

In [2]:
PlaceCellObject = place_cell.run_placeMetrics(calcium_imag,timevector,x_coordinates,y_coordinates)

/home/apolo/Documents/DataAnalysis/Projects/CalciumSignalAnalysis/place_cell_base.py:146: RuntimeWarning: Mean of empty slice
  calcium_mean_occupancy[yy,xx] = np.nanmean(mean_calcium_to_behavior_speed[np.logical_and(check_x_ocuppancy,check_y_ocuppancy)])


File not saved!


In [7]:

plt.figure()
plt.pcolormesh(PlaceCellObject['x_grid'],PlaceCellObject['y_grid'],PlaceCellObject['signalMap'])
plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:

plt.figure()
plt.pcolormesh(PlaceCellObject['x_grid'],PlaceCellObject['y_grid'],PlaceCellObject['signalMapSmoothed'])
plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:

center_x = PlaceCellObject['x_grid'][0:-1] + np.diff(PlaceCellObject['x_grid'])
center_y = PlaceCellObject['y_grid'][0:-1] + np.diff(PlaceCellObject['y_grid'])

# (X,Y) = np.meshgrid(center_x,center_y)

plt.figure()
cnt = plt.contourf(center_x,center_y,PlaceCellObject['signalMapSmoothed'],100)
for c in cnt.collections:
    c.set_edgecolor("face")


plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …